# Tensorboard Copmarison

Another notebook shows how to use wandb and what it can do, this notebook will demonstrate tensorboard features as a compaison.

The content can be found on tf doc [here](https://www.tensorflow.org/tensorboard/get_started).


## Prepare Environment

In [2]:
!pip install tensorflow -q


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


First, we need to load the extension for jupyter notebook.

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
import tensorflow as tf
import datetime

2023-08-14 12:12:34.215354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Clear any logs from previous runs
!rm -rf ./tmp/logs/

## Training Code

Here is a simple model for classifying MNIST dataset into 10 classes.

In [7]:
# Load dataset
mnist = tf.keras.datasets.mnist

# Normalize data
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a simple model
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

11490434/11490434 [==============================] - 4s 0us/step


## Define pipeline and add tensorboard

Keras can support tensorboard via callback, so it's pretty easy to add tensorboard.

You can start a local tensorboard server to open the tensorboard webpage with below command, and monitor training in real time:
```bash
tensorboard --logdir tmp/logs/tb/demo_fit
```

OR, display the tensorboard in notebook after training.

In [10]:
# Init the model layers and compile.
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "tmp/logs/tb_demo_fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# This callback take care of tensorboard
# Addition: histogram_freq=1 will log activation histograms every 1 epoch
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Fit the model and check tensorboard
model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2191 - accuracy: 0.9358 - val_loss: 0.1062 - val_accuracy: 0.9687
Epoch 2/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0970 - accuracy: 0.9699 - val_loss: 0.0763 - val_accuracy: 0.9777
Epoch 3/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0680 - accuracy: 0.9793 - val_loss: 0.0723 - val_accuracy: 0.9775
Epoch 4/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0512 - accuracy: 0.9837 - val_loss: 0.0693 - val_accuracy: 0.9796
Epoch 5/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0421 - accuracy: 0.9862 - val_loss: 0.0779 - val_accuracy: 0.9772


If you didn't start a local tensorboard server described above, you can start one inside this notebook like below:
(Personally, I don't recommend to start within notebook as it's hard to control the booted instance).

In [1]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

No known TensorBoard instances running.


In [4]:
# notebook.start("--logdir tmp/logs/tb_demo_fit")
# notebook.display(height=1000)

# OR
# %tensorboard --logdir tmp/logs/tb_demo_fit

Launching TensorBoard...

Selecting TensorBoard with logdir tmp/logs/tb_demo_fit (started 0:00:00 ago; port 6006, pid 18481).


From the official doc:
A brief overview of the visualizations created in this example and the dashboards (tabs in top navigation bar) where they can be found:

- `Scalars` show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the Time Series or Scalars dashboards.
- `Graphs` help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the Graphs dashboard.
- `Histograms and Distributions` show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the Time Series or Histograms dashboards. Distributions can be found in the Distributions dashboard.